In [2]:
import os
import sys
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
#os.environ['TORCH_USE_CUDA_DSA'] = '1'

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from medpy import io
import six
import pandas as pd
import SimpleITK as sitk
import seaborn as sns
from sklearn.metrics import roc_auc_score
import math

import torch
from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import lightning.pytorch as pl
from torchsummary import summary
import torch.nn.functional as F

import monai
from monai.networks.nets import resnet10, resnet18, resnet50

from torch.utils.data import DataLoader


/u/home/seel/.conda/envs/conda-adlm/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [28]:
%load_ext autoreload
%autoreload 2
sys.path.append('../')
from data_scripts.PI_CAI_Detection import PI_CAI_Detection
#from models.MedNet import resnet10, resnet18, resnet50

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
labels_path='/data1/practical-sose23/morphometric/picai_labels'
image_path='/data1/practical-sose23/morphometric/data'
modalities=["t2w"]
targetVoxelShape= (1.0, 1.0, 3.0) #(0.5, 0.5, 3.0)#(1,1,1) #EXP1: STUPID RESULTS
targetimageSize = [220, 220, 45]#(500,500,50)
multiModel=False
useAI_Segmentation=False
biasFeildCorrection=False
toShuffle = False
batchSize = 10

dataset = PI_CAI_Detection(image_path,labels_path,modalities,\
                            multiModel,useAI_Segmentation,biasFeildCorrection,\
                            targetVoxelShape,targetimageSize)

#train_Lables=[]
#for _,_,batch_labels in loader:
#    train_Lables.extend(batch_labels)

#class_sample_count = np.unique(train_Lables, return_counts=True)[1]
#weight = 1. / class_sample_count
#samples_weight = weight[train_Lables]
#WeightedRandomSampler = utils.data.sampler.WeightedRandomSampler(samples_weight, len(samples_weight))

generator1 = torch.Generator().manual_seed(42)
train, val, test = utils.data.random_split(dataset, [0.6, 0.2, 0.2], generator=generator1)



train_loader = DataLoader(train,batch_size=batchSize,shuffle=toShuffle, num_workers=4)#, sampler=WeightedRandomSampler)
val_loader = DataLoader(val,batch_size=batchSize,shuffle=toShuffle, num_workers=4)
test_loader = DataLoader(test,batch_size=batchSize,shuffle=toShuffle, num_workers=4)


In [31]:
for i,x in enumerate(test_loader):
    view1, view2, label = x
    print(label)

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f16853143a0>
Traceback (most recent call last):
  File "/u/home/seel/.conda/envs/conda-adlm/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/u/home/seel/.conda/envs/conda-adlm/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/u/home/seel/.conda/envs/conda-adlm/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f16853143a0>
Traceback (most recent call last):
  File "/u/home/seel/.conda/envs/conda-adlm/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/u/home/seel/.conda/envs/conda-adlm/lib/p

tensor([0, 0, 1, 0, 1, 0, 0, 0, 0, 0])
tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 1])
tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 1])
tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 1])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 1, 1, 0, 1, 1, 1, 1, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 1, 0, 1, 0, 1, 0, 0, 1, 1])
tensor([1, 1, 0, 0, 0, 1, 0, 0, 0, 1])
tensor([1, 0, 0, 1, 0, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 0, 0, 0, 0, 1, 1, 0])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 1])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 1, 0, 0, 0, 0, 1, 0])
tensor([1, 0, 0, 0, 0, 1, 0, 0, 1, 0])
tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0,

In [36]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(220 * 220 * 45, 1)

    def forward(self, x):
        
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.sigmoid(self.fc1(x))
        x = torch.squeeze(x,1)
        return x


net = Net()

In [37]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [39]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, input2, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,    10] loss: 0.099
[1,    20] loss: 0.115
[1,    30] loss: 0.045
[1,    40] loss: 0.095
[1,    50] loss: 0.060
[1,    60] loss: 0.085


KeyboardInterrupt: 